In [1]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import backtrader as bt

In [2]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.maperiod, self.broker.getvalue()), doprint=True)

In [3]:
mt5.initialize(path = "C:/Program Files/Genial Investimentos MetaTrader 5/terminal64.exe")

True

In [4]:
today = datetime.today().strftime("%Y-%m-%d")
today

'2021-06-23'

In [5]:
df = pd.DataFrame(mt5.copy_rates_from('VALE3', mt5.TIMEFRAME_M2, datetime.today(), 1000))
df['time']=pd.to_datetime(df['time'], unit='s')
df.set_index('time', inplace = True)
df.head()

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2021-06-17 11:18:00,107.30,107.54,107.26,107.45,515,1,172000
2021-06-17 11:20:00,107.45,107.48,107.00,107.28,610,1,213400
2021-06-17 11:22:00,107.28,107.29,107.10,107.13,536,1,111200
2021-06-17 11:24:00,107.12,107.17,107.03,107.13,485,1,119300
2021-06-17 11:26:00,107.12,107.23,107.05,107.19,402,1,253500


In [6]:
cerebro = bt.Cerebro()

In [7]:
# Setting comission
# 0.1% ... divide by 100 to remove the %
cerebro.broker.setcommission(commission=0.001)

In [8]:
data = bt.feeds.PandasData(dataname=df, openinterest=None)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 100000.00


In [9]:
cerebro.addstrategy(TestStrategy)

0

In [10]:
# Run over everything
cerebro.run()

2021-06-23, (MA Period 15) Ending Value 99980.01


In [11]:
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Final Portfolio Value: 99980.01


In [12]:
#cerebro.plot(volume=False)